In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import pyvista as pv
import os
import utils
from stpyvista import stpyvista
from datetime import datetime
import importlib

/opt/homebrew/lib/python3.11/site-packages/stpyvista/__init__.py:18: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("vtk", sizing_mode="stretch_both")


In [3]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [4]:
df_grouped = utils.get_full_training_dataset(df)

In [5]:
df_grouped

tmp     hum    CO2    VOC    vis    IR  \
device_id     date_time_rounded                                                
hka-aqm-am001 2022-09-05 13:00:00  25.050  51.890  993.0  606.5   68.0  11.5   
              2022-09-05 13:30:00  25.110  51.935  864.0  497.0  109.0  18.5   
              2022-09-05 14:00:00  25.345  51.760  658.0  491.5   95.5  14.0   
              2022-09-05 14:30:00  25.580  51.630  642.0  579.0  116.0  19.0   
              2022-09-05 16:00:00  25.710  50.270  617.0  633.0   25.0   9.0   
...                                   ...     ...    ...    ...    ...   ...   
hka-aqm-am308 2023-09-26 22:00:00  27.435  37.415  522.5  957.5    4.0   1.0   
              2023-09-26 22:30:00  27.380  37.510  527.5  958.5    5.5   2.0   
              2023-09-26 23:00:00  27.370  37.580  524.5  949.0    5.5   2.5   
              2023-09-26 23:30:00  27.345  37.630  527.0  955.0    4.0   0.0   
              2023-09-27 00:00:00  27.330  37.650  525.0  951.0    4.0   0.0   

                                   WIFI   BLE   rssi  channel_rssi  ...  \
device_id     date_time_rounded                                     ...   
hka-aqm-am001 2022-09-05 13:00:00   1.0   0.5 -127.5        -127.5  ...   
              2022-09-05 13:30:00   2.0   0.0 -128.5        -128.5  ...   
              2022-09-05 14:00:00   2.5   0.5 -134.0        -134.0  ...   
              2022-09-05 14:30:00   0.0   0.0 -134.0        -134.0  ...   
              2022-09-05 16:00:00   1.0   0.0 -138.0        -138.0  ...   
...                                 ...   ...    ...           ...  ...   
hka-aqm-am308 2023-09-26 22:00:00   3.5  13.5  -74.0         -74.0  ...   
              2023-09-26 22:30:00   1.0   0.0  -73.0         -73.0  ...   
              2023-09-26 23:00:00   1.5   0.0  -72.5         -72.5  ...   
              2023-09-26 23:30:00   3.0  13.5  -75.0         -75.0  ...   
              2023-09-27 00:00:00   0.0   0.0  -70.0         -70.0  ...   

                                   VOC_lag_3  VOC_lag_4  VOC_lag_5  VOC_next  \
device_id     date_time_rounded                                                
hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN     497.0   
              2022-09-05 13:30:00        NaN        NaN        NaN     491.5   
              2022-09-05 14:00:00        NaN        NaN        NaN     579.0   
              2022-09-05 14:30:00      606.5        NaN        NaN       NaN   
              2022-09-05 16:00:00      579.0      491.5      497.0       NaN   
...                                      ...        ...        ...       ...   
hka-aqm-am308 2023-09-26 22:00:00      953.0      953.5      944.0     958.5   
              2023-09-26 22:30:00      950.5      953.0      953.5     949.0   
              2023-09-26 23:00:00      955.0      950.5      953.0     955.0   
              2023-09-26 23:30:00      957.5      955.0      950.5     951.0   
              2023-09-27 00:00:00      958.5      957.5      955.0       NaN   

                                   vis_lag_1  vis_lag_2  vis_lag_3  vis_lag_4  \
device_id     date_time_rounded                                                 
hka-aqm-am001 2022-09-05 13:00:00        NaN        NaN        NaN        NaN   
              2022-09-05 13:30:00       68.0        NaN        NaN        NaN   
              2022-09-05 14:00:00      109.0       68.0        NaN        NaN   
              2022-09-05 14:30:00       95.5      109.0       68.0        NaN   
              2022-09-05 16:00:00        NaN        NaN      116.0       95.5   
...                                      ...        ...        ...        ...   
hka-aqm-am308 2023-09-26 22:00:00        4.0        5.5        5.5        5.5   
              2023-09-26 22:30:00        4.0        4.0        5.5        5.5   
              2023-09-26 23:00:00        5.5        4.0        4.0        5.5   
              2023-09-26 23:30:00        5.5        5.5        4.0        4.0   
        

In [7]:
importlib.reload(utils)
train_dataset, test_dataset, train_loader, test_loader, scaler, y_test, full_preprocessed_df, y_feature_scaler_index = utils.get_data_for_multivarate_sequential_forecast(df)
full_preprocessed_df

training data cutoff:  2023-07-14 05:00:00
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am0

/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_s

feature count:  49
feature count:  49
Training data shape: torch.Size([158137, 5, 49]) torch.Size([158137, 1])
Testing data shape: torch.Size([39792, 5, 49]) torch.Size([39792, 1])


ValueError: too many values to unpack (expected 6)

In [8]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_transformer_model_for_feature(df, d_model=128, nhead=8, num_layers=4, dropout=0.1, batch_size=128, learning_rate=0.0001)

training data cutoff:  2023-07-15 02:45:00


/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[columns_to_s

tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday                     float64
month                       float64
hour_sin                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                       float64
device_id_hka-aqm-am001     float64
device_id_hka-aqm-am002     float64
device_id_hka-aqm-am003a    float64
device_id_hka-aqm-am003b    float64
device_id_hka-aqm-am004     

KeyboardInterrupt: 

In [ ]:
importlib.reload(utils)
model, scaler = utils.create_multivariate_model_for_feature(df, aggregation_level='half_hour')

Training data shape: torch.Size([150050, 47]) torch.Size([150050])
Testing data shape: torch.Size([50023, 47]) torch.Size([50023])
     actual  predicted
0     408.0  477.67099
1     410.0  477.67099
2     406.5  477.67099
3     410.5  477.67099
4     413.5  477.67099
..      ...        ...
123   460.0  477.67099
124   440.0  477.67099
125   432.0  477.67099
126   432.5  477.67099
127   423.5  477.67099

[128 rows x 2 columns]
Epoch 1/1000, Validation Loss: 1.3423


KeyboardInterrupt: 

In [ ]:
train.columns

Index(['tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'WIFI', 'BLE', 'rssi',
       'channel_rssi', 'channel_index', 'spreading_factor', 'bandwidth',
       'f_cnt', 'isHoliday', 'isExamTime', 'weekday', 'month', 'hour_sin',
       'semester_SS23', 'semester_WS22/23', 'semester_WS23/24', 'tmp_lag_1',
       'tmp_lag_2', 'tmp_lag_3', 'tmp_lag_4', 'tmp_lag_5', 'hum_lag_1',
       'hum_lag_2', 'hum_lag_3', 'hum_lag_4', 'hum_lag_5', 'CO2_lag_1',
       'CO2_lag_2', 'CO2_lag_3', 'CO2_lag_4', 'CO2_lag_5', 'CO2_next',
       'VOC_lag_1', 'VOC_lag_2', 'VOC_lag_3', 'VOC_lag_4', 'VOC_lag_5',
       'vis_lag_1', 'vis_lag_2', 'vis_lag_3', 'vis_lag_4', 'vis_lag_5',
       'CO2_next_scaled'],
      dtype='object')

In [ ]:
model, scaler =